In [332]:
import pandas as pd
from datetime import datetime
from geopy import distance
import numpy as np

# Load data

In [ ]:
july = pd.read_csv("202207-divvy-tripdata.csv")
august = pd.read_csv("202208-divvy-tripdata.csv")
september = pd.read_csv("202209-divvy-tripdata.csv")
october = pd.read_csv("202210-divvy-tripdata.csv")
november = pd.read_csv("202211-divvy-tripdata.csv")
december = pd.read_csv("202212-divvy-tripdata.csv")
january = pd.read_csv("202301-divvy-tripdata.csv")
february  = pd.read_csv("202302-divvy-tripdata.csv")
march = pd.read_csv("202303-divvy-tripdata.csv")
april = pd.read_csv("202304-divvy-tripdata.csv")
may = pd.read_csv("202305-divvy-tripdata.csv")
june = pd.read_csv("202306-divvy-tripdata.csv")

## Merge the data to one dataframe

In [9]:
df = pd.concat([july, august, september, october, november, december, january, february, march, april, may, june], axis=0)
df.to_csv"all.csv", index=False()

# Overview

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779444 entries, 0 to 5779443
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 573.2+ MB


# Data cleaning

## Missing values

In [21]:
df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    857860
start_station_id      857992
end_station_name      915655
end_station_id        915796
start_lat                  0
start_lng                  0
end_lat                 5795
end_lng                 5795
member_casual              0
dtype: int64

## Data validation

### Check if the length of ride id is equal to 16

In [23]:
print(f"There are {(-df['ride_id'].apply(lambda x: len(x)==16)).sum()} id does not have length of 16")

There are 0 id does not have length of 16


### Check the categories of bikes 

In [24]:
print(f"Types of bikes: {df['rideable_type'].unique()}")

Types of bikes: ['classic_bike' 'electric_bike' 'docked_bike']


### Check if started_at, ended_at are in correct time format

In [25]:
df["started_at"] = pd.to_datetime(df["started_at"], errors='coerce')
print(f"There are {(-df['started_at'].dt.time.notna()).sum()} values in started_at that are not in time format")
df["ended_at"] = pd.to_datetime(df["ended_at"], errors='coerce')
print(f"There are {(-df['ended_at'].dt.time.notna()).sum()} values in ended_at that are not in time format")

There are 0 values in started_at that are not in time format
There are 0 values in ended_at that are not in time format


### Check latitude and longitude

In [26]:
print(f"There are {(-df['start_lat'].between(-90, 90)).sum()} values in start_lat that do not match the format")
print(f"There are {(-df['start_lng'].between(-180, 180)).sum()} values in start_lng that do not match the format")
print(f"There are {(-df['end_lat'].between(-90, 90)).sum()} values in end_lat that do not match the format")
print(f"There are {(-df['end_lng'].between(-180, 180)).sum()} values in end_lng that do not match the format")

There are 0 values in start_lat that do not match the format
There are 0 values in start_lng that do not match the format
There are 5795 values in end_lat that do not match the format
There are 5795 values in end_lng that do not match the format


In [28]:
# the values that do not match the format are nan values
print(f"There are {df['end_lat'][-df['end_lat'].between(-180, 180)].notnull().sum()} not null end_lat values that do not match the format")
print(f"There are {df['end_lng'][-df['end_lng'].between(-180, 180)].notnull().sum()} not null end_lng valuest that do not match the format")

There are 0 not null end_lat values that do not match the format
There are 0 not null end_lng valuest that do not match the format


### Check the categories of member_casual

In [29]:
print(f"Categories of member_casual: {df['member_casual'].unique()}")

Categories of member_casual: ['member' 'casual']


### Check if there is two stations with same id but different name (misspell...)

In [31]:
# create a dataframe with a lists of stations
start = df.loc[:,["start_station_name",	"start_station_id", "start_lat", "start_lng"]]
end = df.loc[:,["end_station_name", "end_station_id", "end_lat", "end_lng"]]
stations_df = pd.concat([start.rename(columns={'start_station_name':'name', 'start_station_id': 'id', 'start_lat': 'lat', 'start_lng': 'lng'}), end.rename(columns={'end_station_name':'name', 'end_station_id': 'id', 'end_lat': 'lat', 'end_lng': 'lng'})])

# find if a id matches two different station names
result = stations_df.groupby('id')['name'].nunique() > 1
ids_with_different_names = stations_df[stations_df['id'].isin(result[result].index)]

# keep only one instance
stations_df_drop = ids_with_different_names[['name', 'id']].drop_duplicates()

# get the latitude and longitude for stations by taking the mode
stations_df_lat_lng = stations_df.groupby(['name', 'id'])['lat', 'lng'].agg(lambda x: pd.Series.mode(x)[0])

# get a list of ids
id_list = stations_df['id'].unique()

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/1727141480.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stations_df_lat_lng = stations_df.groupby(['name', 'id'])['lat', 'lng'].agg(lambda x: pd.Series.mode(x)[0])


In [32]:
stations_df_drop

,name,id
1,Buckingham Fountain (Temp),15541
25,Bissell St & Armitage Ave,13059
34,Public Rack - Cicero Ave & Wellington Ave,546
40,University Library (NU),605
77,Knox Ave & Montrose Ave,592
...,...,...
4060527,Public Rack - Michigan Ave & 102nd St,630
4072006,Public Rack - Keeler Ave & Madison St S,853
4705314,Public Rack - Plainfield Ave & Irving Park Rd,576
4912379,Public Rack - Parnell Ave & 119th St,587


In [34]:
stations_df_lat_lng

,,lat,lng
name,id,,
10101 S Stony Island Ave,922,41.710000,-87.580000
111th St - Morgan Park Metra,682,41.690000,-87.670000
2112 W Peterson Ave,KA1504000155,41.991178,-87.683593
410,410,41.900000,-87.690000
532 E 43rd St,913,41.820000,-87.610000
...,...,...,...
Woodlawn Ave & Lake Park Ave,KA1503000065,41.814093,-87.597005
Yale Ave & 119th St,588,41.680000,-87.630000
Yates Blvd & 75th St,KA1503000024,41.758768,-87.566440


### Two examples

In [69]:
stations_df_drop[stations_df_drop['id']=='922']


,name,id
1361808,10101 S Stony Island Ave,922
3210529,Public Rack - 10101 S Stony Island Ave,922


In [88]:
stations_df_lat_lng.loc[('10101 S Stony Island Ave', '922')], stations_df_lat_lng.loc[('Public Rack - 10101 S Stony Island Ave', '922')] # same id same location 

(lat    41.71
 lng   -87.58
 Name: (10101 S Stony Island Ave, 922), dtype: float64,
 lat    41.71
 lng   -87.58
 Name: (Public Rack - 10101 S Stony Island Ave, 922), dtype: float64)

In [110]:
stations_df_drop[stations_df_drop['id']=='13059'] 

,name,id
25,Bissell St & Armitage Ave,13059
222,Sheridan Rd & Argyle St,13059


In [111]:
stations_df_lat_lng.loc[('Bissell St & Armitage Ave', '13059')], stations_df_lat_lng.loc[('Sheridan Rd & Argyle St', '13059')] # same id different location 

(lat    41.918018
 lng   -87.652182
 Name: (Bissell St & Armitage Ave, 13059), dtype: float64,
 lat    41.973287
 lng   -87.654760
 Name: (Sheridan Rd & Argyle St, 13059), dtype: float64)

In [109]:
stations_df['id'].str.startswith("A").value_counts() # no id start with A

False    9785100
Name: id, dtype: int64

In [158]:
modify = {"change_name": {}, "change_id": {}} # change_name: distance is less than 50 meters, so we only need to change name, change_id: distance is greater than 50 meters
distance_list = []
for id, name in stations_df_drop.groupby('id', as_index=False)['name']:
    distances = distance.distance(stations_df_lat_lng.loc[(name.values[0], id)].values, stations_df_lat_lng.loc[(name.values[1], id)].values).km
    distance_list.append(distances)
    if distances <= 0.5:
        modify["change_name"][id] = name.values[0]
    else:
        modify["change_id"][name.values[0]] = "A" + id + "-1" # create new id 

In [159]:
len(modify['change_name']), len(modify['change_id'])

(223, 97)

In [160]:
df1 = df.copy()

In [161]:
for item in modify['change_name'].items():
    df1.loc[df1['start_station_id'] == item[0], 'start_station_name'] = item[1]
    df1.loc[df1['end_station_id'] == item[0], 'end_station_name'] = item[1]
for item in modify['change_id'].items():
    df1.loc[df1['start_station_name'] == item[0], 'start_station_id'] = item[1]
    df1.loc[df1['end_station_name'] == item[0], 'end_station_id'] = item[1]

### Check the modified df1

In [162]:
df[df['end_station_id']=='922']['end_station_name'].unique()

array(['10101 S Stony Island Ave',
       'Public Rack - 10101 S Stony Island Ave'], dtype=object)

In [163]:
df1[df1['end_station_id']=='922']['end_station_name'].unique()

array(['10101 S Stony Island Ave'], dtype=object)

In [164]:
df1[df['end_station_name']=='Public Rack - 10101 S Stony Island Ave']

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3210529,816E85DF05085E85,electric_bike,2022-12-01 19:12:33,2022-12-01 19:19:10,Maryland Ave & 104th St,20134,10101 S Stony Island Ave,922,41.71,-87.6,41.71,-87.58,casual


In [165]:
df[df['end_station_id']=='13059']['end_station_name'].unique()

array(['Bissell St & Armitage Ave', 'Sheridan Rd & Argyle St'],
      dtype=object)

In [166]:
df1[df1['end_station_id']=='13059']['end_station_name'].unique()

array(['Sheridan Rd & Argyle St'], dtype=object)

In [167]:
df1[df['end_station_name']=='Bissell St & Armitage Ave']['end_station_id'].unique()

array(['A13059-1'], dtype=object)

# Deal with missing values
As remembered, we have 
- start_station_name:    857860
- start_station_id:     857992
- end_station_name:      915655
- end_station_id:        915796
- end_lat:                5795
- end_lng:                 5795

# First deal with end_lat and end_lng

In [168]:
df_end_na = df1[df1['end_lat'].isna() & df1['end_lng'].isna()]
df_end_na

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
36126,87BB03841B82D504,docked_bike,2022-07-02 07:08:11,2022-07-03 08:08:12,Montrose Harbor,TA1308000012,NaN,NaN,41.963982,-87.638181,NaN,NaN,casual
36267,25A007F2C8F64C47,docked_bike,2022-07-31 03:43:11,2022-08-01 04:43:11,Wilton Ave & Belmont Ave,TA1307000134,NaN,NaN,41.940180,-87.653040,NaN,NaN,casual
36298,3214D4171EFEABB8,docked_bike,2022-07-04 14:48:04,2022-07-06 04:35:55,Broadway & Granville Ave,15571,NaN,NaN,41.994780,-87.660285,NaN,NaN,casual
36335,56CDD157E2A5FA9E,docked_bike,2022-07-09 17:30:57,2022-07-11 04:53:37,Ritchie Ct & Banks St,KA1504000134,NaN,NaN,41.906866,-87.626217,NaN,NaN,casual
36357,EE1476DF7A8D7D4B,docked_bike,2022-07-03 17:33:33,2022-07-03 17:50:34,Shedd Aquarium,15544,NaN,NaN,41.867226,-87.615355,NaN,NaN,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779241,C7B545B3BE23D680,classic_bike,2023-06-11 00:03:01,2023-06-12 01:02:48,Leavitt St & Addison St,KA1504000143,NaN,NaN,41.946655,-87.683359,NaN,NaN,casual
5779315,6445BF0AC9810E43,docked_bike,2023-06-23 18:34:00,2023-06-23 19:07:33,Leavitt St & Addison St,KA1504000143,NaN,NaN,41.946655,-87.683359,NaN,NaN,casual
5779322,135D6E16C06A802E,classic_bike,2023-06-19 14:02:04,2023-06-20 15:01:43,Paulina St & Montrose Ave,TA1309000021,NaN,NaN,41.961507,-87.671387,NaN,NaN,casual
5779428,AF036A8A8EBF731E,docked_bike,2023-06-10 11:56:08,2023-06-15 04:57:09,Clark St & Lincoln Ave,13179,NaN,NaN,41.915689,-87.634600,NaN,NaN,casual


In [169]:
(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna()).sum()

5785

There are 10 instances having empty end_lat and end_lng but with end_station_name and end_station_id

In [204]:
df_end_na[-(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna())]

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/3948280603.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_end_na[-(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna())]


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
5166835,685DB4D7A6AF6CAE,docked_bike,2023-06-29 17:35:41,2023-06-29 17:45:38,Sedgwick St & Huron St,TA1307000062,Halsted St & Fulton St,23003,41.894666,-87.638437,NaN,NaN,casual
5205953,E4AB9F672ECD0966,docked_bike,2023-06-29 14:50:04,2023-06-29 15:06:49,Wabash Ave & Grand Ave,TA1307000117,Halsted St & Fulton St,23003,41.891466,-87.626761,NaN,NaN,casual
5206917,B0A0B0C83B363BC3,docked_bike,2023-06-23 16:06:21,2023-06-23 16:16:55,Cottage Grove Ave & 63rd St,KA1503000054,Stony Island Ave & 63rd St,653B,41.780531,-87.605970,NaN,NaN,casual
5253449,2C35DCA44370EAD8,docked_bike,2023-06-30 18:45:11,2023-06-30 19:06:34,Damen Ave & Foster Ave,KA1504000149,Lincoln Ave & Byron St,23002,41.975615,-87.679459,NaN,NaN,casual
5254449,D19CA7CA83B28F88,docked_bike,2023-06-29 17:36:45,2023-06-29 18:02:39,Manor Ave & Leland Ave,KA1504000127,Lincoln Ave & Byron St,23002,41.965875,-87.700825,NaN,NaN,casual
5256454,1005EAF8E1C29D5C,docked_bike,2023-06-24 13:56:09,2023-06-24 14:18:23,Clark St & Bryn Mawr Ave,KA1504000151,Lincoln Ave & Byron St,23002,41.983593,-87.669154,NaN,NaN,casual
5259381,B34C24AD17CCB667,docked_bike,2023-06-22 22:32:59,2023-06-22 23:09:50,Green St & Randolph St,13053,Halsted St & Fulton St,23003,41.883181,-87.648725,NaN,NaN,casual
5259389,64D120B77FA6F330,docked_bike,2023-06-22 22:32:41,2023-06-22 23:09:48,Green St & Randolph St,13053,Halsted St & Fulton St,23003,41.883181,-87.648725,NaN,NaN,casual
5259519,92008CFA88E93F44,docked_bike,2023-06-22 22:33:03,2023-06-22 23:09:44,Green St & Randolph St,13053,Halsted St & Fulton St,23003,41.883181,-87.648725,NaN,NaN,casual
5262117,84106BA64096E4EC,docked_bike,2023-06-30 14:06:15,2023-06-30 14:09:53,Clinton St & Lake St,13021,Halsted St & Fulton St,23003,41.885637,-87.641823,NaN,NaN,casual


In [239]:
df_end_na_with_station = df_end_na[-(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna())]

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/3877382732.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_end_na_with_station = df_end_na[-(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna())]


### Solution
The solution is to find the corresponded station_id and fill the empty cells

## Create a list of all stations and their latitude and longitude

In [340]:
start = df1.loc[:,["start_station_name","start_station_id", "start_lat", "start_lng"]]
end = df1.loc[:,["end_station_name", "end_station_id", "end_lat", "end_lng"]]
stations_df = pd.concat([start.rename(columns={'start_station_name':'name', 'start_station_id': 'id', 'start_lat': 'lat', 'start_lng': 'lng'}), end.rename(columns={'end_station_name':'name', 'end_station_id': 'id', 'end_lat': 'lat', 'end_lng': 'lng'})])
stations_df_lat_lng = stations_df.groupby(['name', 'id'])['lat', 'lng'].agg(lambda x: pd.Series.mode(x)[0])

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/3352569540.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stations_df_lat_lng = stations_df.groupby(['name', 'id'])['lat', 'lng'].agg(lambda x: pd.Series.mode(x)[0])


In [232]:
stations_df_lat_lng.loc[('Halsted St & Fulton St','23003')]

lat    41.89
lng   -87.65
Name: (Halsted St & Fulton St, 23003), dtype: float64

In [242]:
for index, row in df_end_na_with_station.iterrows():
   lat_lng = stations_df_lat_lng.loc[(row['end_station_name'],row['end_station_id'])]
   df_end_na_with_station.loc[index, 'end_lat'] = lat_lng[0]
   df_end_na_with_station.loc[index, 'end_lng'] = lat_lng[1]

In [245]:
df1.update(df_end_na_with_station)

In [253]:
df_end_na = df1[df1['end_lat'].isna() & df1['end_lng'].isna()]
df_end_na

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
36126,87BB03841B82D504,docked_bike,2022-07-02 07:08:11,2022-07-03 08:08:12,Montrose Harbor,TA1308000012,NaN,NaN,41.963982,-87.638181,NaN,NaN,casual
36267,25A007F2C8F64C47,docked_bike,2022-07-31 03:43:11,2022-08-01 04:43:11,Wilton Ave & Belmont Ave,TA1307000134,NaN,NaN,41.940180,-87.653040,NaN,NaN,casual
36298,3214D4171EFEABB8,docked_bike,2022-07-04 14:48:04,2022-07-06 04:35:55,Broadway & Granville Ave,15571,NaN,NaN,41.994780,-87.660285,NaN,NaN,casual
36335,56CDD157E2A5FA9E,docked_bike,2022-07-09 17:30:57,2022-07-11 04:53:37,Ritchie Ct & Banks St,KA1504000134,NaN,NaN,41.906866,-87.626217,NaN,NaN,casual
36357,EE1476DF7A8D7D4B,docked_bike,2022-07-03 17:33:33,2022-07-03 17:50:34,Shedd Aquarium,15544,NaN,NaN,41.867226,-87.615355,NaN,NaN,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5779241,C7B545B3BE23D680,classic_bike,2023-06-11 00:03:01,2023-06-12 01:02:48,Leavitt St & Addison St,KA1504000143,NaN,NaN,41.946655,-87.683359,NaN,NaN,casual
5779315,6445BF0AC9810E43,docked_bike,2023-06-23 18:34:00,2023-06-23 19:07:33,Leavitt St & Addison St,KA1504000143,NaN,NaN,41.946655,-87.683359,NaN,NaN,casual
5779322,135D6E16C06A802E,classic_bike,2023-06-19 14:02:04,2023-06-20 15:01:43,Paulina St & Montrose Ave,TA1309000021,NaN,NaN,41.961507,-87.671387,NaN,NaN,casual
5779428,AF036A8A8EBF731E,docked_bike,2023-06-10 11:56:08,2023-06-15 04:57:09,Clark St & Lincoln Ave,13179,NaN,NaN,41.915689,-87.634600,NaN,NaN,casual


In [258]:
df_end_na[-(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna())]

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/3948280603.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_end_na[-(df1['end_station_id'].isna() & df1['end_station_name'].isna() & df1['end_lat'].isna() & df1['end_lng'].isna())]


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual


## Fill the empty station names and end_station ids by lat and lng

In [362]:
# stations_df.groupby(['name', 'id'])['lat', 'lng'].agg(lambda x: pd.Series.mode(x)[0])
lat_lng_table = stations_df.dropna()
lat_lng_table = lat_lng_table.groupby(['lat', 'lng'])['name', 'id'].first()
dict_lat_lng = lat_lng_table.to_dict(orient='index')

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/1075009093.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  lat_lng_table = lat_lng_table.groupby(['lat', 'lng'])['name', 'id'].first()


In [438]:
df_start_station_na = df1[(df1['start_station_name'].isna() & df1['start_station_id'].isna())]
df_end_station_na = df1[(df1['end_station_name'].isna() & df1['end_station_id'].isna())]
len(df_start_station_na), len(df_end_station_na)

(857860, 915655)

In [439]:
def update_station_lat_lng(x, start_end, name_or_id):
    try:
        if start_end: # start=1, end=0
            return dict_lat_lng[(x['start_lat'], x['start_lng'])][name_or_id]
        return dict_lat_lng[(x['end_lat'], x['end_lng'])][name_or_id]
    except KeyError:
        pass

df_start_station_na['start_station_name'] = df_start_station_na.apply(lambda x: update_station_lat_lng(x, 1, 'name'), axis=1)
df_start_station_na['start_station_id'] = df_start_station_na.apply(lambda x: update_station_lat_lng(x, 1, 'id'), axis=1)

df_end_station_na['end_station_name'] = df_end_station_na.apply(lambda x: update_station_lat_lng(x, 0, 'name'), axis=1)
df_end_station_na['end_station_id'] = df_end_station_na.apply(lambda x: update_station_lat_lng(x, 0, 'id'), axis=1)
    

/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/3006134399.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_start_station_na['start_station_name'] = df_start_station_na.apply(lambda x: update_station_lat_lng(x, 1, 'name'), axis=1)
/var/folders/95/lczm9_vd3899jm27hx7pnk580000gn/T/ipykernel_10248/3006134399.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_start_station_na['start_station_id'] = df_start_station_na.apply(lambda x: update_station_lat_lng(x, 1, 'id'), axis=1)
/var/fo

In [440]:
df_start_station_na[["start_station_name","start_station_id"]].isnull().sum()

start_station_name    520884
start_station_id      520884
dtype: int64

In [441]:
df_end_station_na[["end_station_name","end_station_id"]].isnull().sum()

end_station_name    561104
end_station_id      561104
dtype: int64

In [442]:
df1.update(df_start_station_na)

In [443]:
df1.update(df_end_station_na)

In [444]:
df1.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    520884
start_station_id      521016
end_station_name      561104
end_station_id        561245
start_lat                  0
start_lng                  0
end_lat                 5785
end_lng                 5785
member_casual              0
dtype: int64

In [445]:
df1.to_csv("all_cleaned.csv", index=False)